In [1]:
import os,sys,shutil
import numpy as np
import matplotlib.pyplot as plt
import time
from mpm_rect import mpm_rect
from mpm_circ import mpm_circ
sys.path.append(os.getcwd()+'/build')
import MPM_2D
import math
from tqdm import tqdm, trange

In [2]:
#Defining the grid
x_0 = 0; y_0 = 0;                                #Origin of X and Y Axis
lx = 0.12; ly = 0.08;                            #Length of grid in X and Y direction
nx = 24; ny = 16;                                #Number of cells in X and Y direction

Grid = MPM_2D.MPM_Grid(L=[lx, ly], N=[nx, ny], O=[x_0, y_0])
NoNodes = Grid.NoNodes

In [3]:
#Defining the water body
visc = 0.01; density_l = 1000; density_h = 1500; bulk_mod = 2e5;
MPV_1 =  mpm_rect([0,0], [0.06,0.06], [30,30], 2)
Water_l = MPM_2D.MPM_Fluid(visc, density_l, bulk_mod, MPV_1)

MPV_2 =  mpm_rect([0.06,0], [0.06,0.06], [30,30], 2)
Water_h = MPM_2D.MPM_Fluid(visc, density_h, bulk_mod, MPV_2)

Water_l.MPC()[:,18] = -9.81        #Gravity
Water_h.MPC()[:,18] = -9.81        #Gravity

In [5]:
dx = lx/nx; dy = ly/ny
# fig, ax = plt.subplots(1, 1, figsize = (5, 5))                                  #creating axis
# ax.set_xlim(x_0,x_0+lx); ax.set_ylim(y_0,y_0+ly)                                #set limits
# #for i in range (nx+1):
#     #ax.plot([x_0+dx*i,x_0+dx*i],[y_0,ly+y_0], c=(0.9, 0.9, 0.9, 1.0) )          #Plot X lines
#     #for j in range (ny+1):
#         #ax.plot([x_0,lx+x_0],[y_0+dy*j,y_0+dy*j], c=(0.9, 0.9, 0.9, 1.0) )      #Plot Y lines
# ax.scatter(Water_l.MPC()[:,0], Water_l.MPC()[:,1] ,s=Water_l.MPC()[:,4]*1e5,c='c')
# ax.scatter(Water_h.MPC()[:,0], Water_h.MPC()[:,1] ,s=Water_h.MPC()[:,4]*1e5,c='b')
# ax.set_aspect(aspect='equal')

In [6]:
#Prepare postprocessing
plotstarget_directory = os.path.join(os.getcwd(), "mixture_plots")

#Create a new directory to store the plots (delete when there)
if os.path.exists(plotstarget_directory) and os.path.isdir(plotstarget_directory):
    shutil.rmtree(plotstarget_directory)
os.mkdir(plotstarget_directory)
print("Plots will be saved to : ", plotstarget_directory)

def save_plot(current_time):
    dx = lx/nx; dy = ly/ny
    fig, ax = plt.subplots(1, 1, figsize = (5, 5))                                  #creating axis
    ax.set_xlim(x_0,x_0+lx); ax.set_ylim(y_0,y_0+ly)                                #set limits
    #for i in range (nx+1):
         #ax.plot([x_0+dx*i,x_0+dx*i],[y_0,ly+y_0], c=(0.9, 0.9, 0.9, 1.0) )          #Plot X lines
         #for j in range (ny+1):
             #ax.plot([x_0,lx+x_0],[y_0+dy*j,y_0+dy*j], c=(0.9, 0.9, 0.9, 1.0) )      #Plot Y lines
    ax.scatter(Water_l.MPC()[:,0], Water_l.MPC()[:,1] ,s=Water_l.MPC()[:,4]*1e5,c='c')
    ax.scatter(Water_h.MPC()[:,0], Water_h.MPC()[:,1] ,s=Water_h.MPC()[:,4]*1e5,c='b')
    ax.set_aspect(aspect='equal')
    filename = 'Mixture_'+str(current_time)+'.png'
    fig.savefig(os.path.join(plotstarget_directory, filename), dpi=400, bbox_inches="tight")
    plt.close()                                                                     #prevent plotting in jupyter notebook
    return

Plots will be saved to :  /home/karthik/cpp/mpm_2d_cpp/mixture_plots


In [7]:
#Running through time steps
t = 0                                             #start time
T = 2; dt = 1e-6                                  #Total time and time step
alpha = 0                                         #damping factor
NoS = int(T/dt)                                   #Number of time steps
noMicroSteps = int(10000)                         #Incremental time step for solver
noMacroSteps = int(NoS/noMicroSteps)              #Number of vtk files

FluidSolver = MPM_2D.MPM_Solver()
FluidSolver.addBC(lambda X: True if (X[1] == 0.0) else False, vy=0.0)
FluidSolver.addBC(lambda X: True if (X[0] == 0.0) else False, vx=0.0)
FluidSolver.addBC(lambda X: True if (X[0] == 0.12) else False, vx=0.0)
Bodies = [Water_l, Water_h]

start_time = time.time()
save_plot(0)
for step in trange (noMacroSteps):
    FluidSolver.Solve(Grid, Bodies, noMicroSteps, dt, alpha)
    save_plot(step+1)
print("--- %s seconds ---" % (time.time() - start_time))

100%|██████████| 200/200 [1:20:37<00:00, 24.19s/it]--- 4838.506639957428 seconds ---

